In [1]:
import $ivy.`com.typesafe.akka::akka-http:10.2.1`
import $ivy.`com.typesafe.akka::akka-http-spray-json:10.2.1`
import $ivy.`com.typesafe.akka::akka-actor-typed:2.6.10`
import $ivy.`com.typesafe.akka::akka-stream:2.6.10`
import $ivy.`ch.qos.logback:logback-classic:1.2.3`

import $ivy.$                                    

import $ivy.$                                               

import $ivy.$                                           

import $ivy.$                                      

import $ivy.$                                     

In [2]:
repl.pprinter() = repl.pprinter().copy(defaultHeight = 15 )

In [75]:
import akka.stream._, scaladsl._, Tcp._
import akka.{ Done, NotUsed }
import akka.actor.ActorSystem
import akka.util.ByteString
import scala.concurrent._
import scala.concurrent.duration._
import java.nio.file.Paths
import akka.NotUsed
import akka.stream.scaladsl.{BidiFlow, Source}
import akka.stream.stage.{GraphStage, GraphStageLogic, InHandler, OutHandler}
import akka.stream.{Attributes, BidiShape, Inlet, Outlet}
import akka.util.ByteString
import akka.event._


import akka.stream._, scaladsl._, Tcp._

import akka.{ Done, NotUsed }

import akka.actor.ActorSystem

import akka.util.ByteString

import scala.concurrent._

import scala.concurrent.duration._

import java.nio.file.Paths

import akka.NotUsed

import akka.stream.scaladsl.{BidiFlow, Source}

import akka.stream.stage.{GraphStage, GraphStageLogic, InHandler, OutHandler}

import akka.stream.{Attributes, BidiShape, Inlet, Outlet}

import akka.util.ByteString

import akka.event._


In [4]:
implicit val system = ActorSystem("QuickStart")

22:45:57.410 [QuickStart-akka.actor.default-dispatcher-5] INFO akka.event.slf4j.Slf4jLogger - Slf4jLogger started


system: ActorSystem = akka://QuickStart

In [96]:

case class IdentityBidy()
  extends GraphStage[BidiShape[String, ByteString, ByteString, ByteString]] {

  override def toString: String = "SeedlinkProtocol"

  val inC: Inlet[String] = Inlet("inC")
  val outC: Outlet[ByteString] = Outlet("outC")
  val inS: Inlet[ByteString] = Inlet("inS")
  val outS: Outlet[ByteString] = Outlet("outS")

  override val shape = BidiShape(inC, outC, inS, outS)

  override def createLogic(inheritedAttributes: Attributes): GraphStageLogic =
    new GraphStageLogic(shape) {
      setHandler(inC, new InHandler {
        override def onPush(): Unit =
          push(outC, ByteString(grab(inC)))
        override def onUpstreamFinish(): Unit = {
        }
      })

      setHandler(outC, new OutHandler {
        override def onPull(): Unit =
          if (!isClosed(inC)) pull(inC)
      })

      setHandler(inS, new InHandler {
        override def onPush(): Unit = 
          push(outS, grab(inS))
      })

      setHandler(outS, new OutHandler {
        override def onPull(): Unit = 
          pull(inS)
      })
    }
}

object IdentityBidy{

  val bidy: BidiFlow[String, ByteString, ByteString, ByteString, NotUsed] =
    BidiFlow.fromGraph(IdentityBidy())

}

defined class IdentityBidy
defined object IdentityBidy

In [120]:
val okServer: Flow[ByteString, ByteString, NotUsed] = 
    Flow[ByteString].flatMapConcat{ c => 
        println(c.utf8String.trim == "end")
        if (c.utf8String.trim == "end")
            Source(List("1","2","3","4"))
        else Source.single("ok")
    }.map(ByteString.apply)

okServer: Flow[ByteString, ByteString, NotUsed] = Flow(FlowShape(Map.in(1060423012),Map.out(1943187137)))

In [121]:
Source(List("a", "b", "c", "end"))
    .via(IdentityBidy.bidy.join(okServer))
    .log("bidi").withAttributes(Attributes.logLevels(
        onElement = Logging.WarningLevel, 
        onFinish = Logging.WarningLevel, 
        onFailure = Logging.WarningLevel))
    .runWith(Sink.seq) // .runWith(Sink.foreach(bs => print(bs.utf8String)))

false
false
false
11:33:03.634 [QuickStart-akka.actor.default-dispatcher-70] WARN akka.stream.Materializer - [bidi] Element: ByteString(111, 107)
11:33:03.634 [QuickStart-akka.actor.default-dispatcher-70] WARN akka.stream.Materializer - [bidi] Element: ByteString(111, 107)
true
11:33:03.634 [QuickStart-akka.actor.default-dispatcher-70] WARN akka.stream.Materializer - [bidi] Element: ByteString(111, 107)


res120: Future[Seq[ByteString]] = [running]

In [122]:

case class IdentityBidy2()
  extends GraphStage[BidiShape[String, ByteString, ByteString, ByteString]] {

  override def toString: String = "SeedlinkProtocol"

  val inC: Inlet[String] = Inlet("inC")
  val outC: Outlet[ByteString] = Outlet("outC")
  val inS: Inlet[ByteString] = Inlet("inS")
  val outS: Outlet[ByteString] = Outlet("outS")

  override val shape = BidiShape(inC, outC, inS, outS)

  override def createLogic(inheritedAttributes: Attributes): GraphStageLogic =
    new GraphStageLogic(shape) {
      var command: Option[String] = None
        
      var transmission: Boolean = false

      override def preStart(): Unit = {
        pull(inC)
      }

      setHandler(inC, new InHandler {
        override def onPush(): Unit = {
            val elem = grab(inC)
            println(s"inC: onPush $elem")
            if (elem.trim == "end") transmission = true
            push(outC, ByteString(elem))
        }
          
        override def onUpstreamFinish(): Unit = ()
      })

      setHandler(outC, new OutHandler {
        override def onPull(): Unit =
          println(s"ouC: onPull")
      })

      setHandler(inS, new InHandler {
        override def onPush(): Unit = {
          /*if (transmission) push(outS, grab(inS))
          else if (grab(inS).utf8String == "OK\r\n") ()
          else fail(outS, new Exception(s"error received for command ${command.get}"))
          command = None
          pull(inC)*/
            val elem = grab(inS)
            println(s"inS: onPush $elem")
            if (transmission) push(outS, elem)
            else pull(inC)
        }
      })

      setHandler(outS, new OutHandler {
        override def onPull(): Unit = {
            println("ouS: onPull")  
            pull(inS)
        }
      })
    }
}

object IdentityBidy2{

  val bidy: BidiFlow[String, ByteString, ByteString, ByteString, NotUsed] =
    BidiFlow.fromGraph(IdentityBidy2())

}

defined class IdentityBidy2
defined object IdentityBidy2

In [131]:
Source(List("a", "b", "c", "end"))
.log("source").withAttributes(Attributes.logLevels(
        onElement = Logging.WarningLevel, 
        onFinish = Logging.WarningLevel, 
        onFailure = Logging.WarningLevel))
    .via(IdentityBidy2.bidy.join(okServer))
.log("after").withAttributes(Attributes.logLevels(
        onElement = Logging.WarningLevel, 
        onFinish = Logging.WarningLevel, 
        onFailure = Logging.WarningLevel))
    .runWith(Sink.foreach(bs => print(bs.utf8String)))

ouS: onPull
ouC: onPull
inC: onPush a
false
inS: onPush ByteString(111, 107)
ouC: onPull
11:34:32.346 [QuickStart-akka.actor.default-dispatcher-73] WARN akka.stream.Materializer - [source] Element: a
inC: onPush b
false
11:34:32.346 [QuickStart-akka.actor.default-dispatcher-73] WARN akka.stream.Materializer - [source] Element: b


res130: Future[Done] = [running]

In [42]:
Source(List(1)).flatMapConcat{ _ => Source.fromIterator(() => Iterator.iterate(0)(_+1)) }

res41: Source[Int, NotUsed] = Source(SourceShape(flatten.out(449688692)))

In [ ]:
res4.runForeach(println)

In [ ]:
res4.toMat(Sink.head)(Keep.right)

In [ ]:
res7.run()

In [ ]:
Iterator.iterate(0)(_+1)


In [ ]:
res4.take(10).toList

In [ ]:
val server = Flow[Int].mapConcat{ i => 
    if (i<3) List(i) 
    else Iterator.iterate(i)(_+1).take(10).toList
}

In [ ]:
val server = Flow[Int].flatMapConcat{ i => 
    if (i<3) Source.single(i) 
    else Source.fromIterator(() => Iterator.iterate(i)(_+1))
}

In [ ]:
val server = Flow[Int].flatMapMerge(1, i => 
    if (i<3) Source.single(i) 
    else Source.fromIterator(() => Iterator.iterate(i)(_+1))
)

In [ ]:
Source(List(1,2,3)).via(server).runForeach(println)

# Stateful maps

In [ ]:
val letterAndIndex = Source("a" :: "b" :: "c" :: "d" :: Nil).statefulMapConcat { () =>
  var counter = 0L

  // we return the function that will be invoked for each element
  { element =>
    counter += 1
    // we return an iterable with the single element
    (element, counter) :: Nil
  }
}


In [ ]:

letterAndIndex.runForeach(println)

In [ ]:
val fruitsAndDeniedCommands = Source(
  "banana" :: "pear" :: "orange" :: "deny:banana" :: "banana" :: "pear" :: "banana" :: Nil)


In [ ]:
val denyFilterFlow = Flow[String].statefulMapConcat { () =>
  var denyList = Set.empty[String]

  { element =>
    if (element.startsWith("deny:")) {
      denyList += element.drop(5)
      Nil // no element downstream when adding a deny listed keyword
    } else if (denyList(element)) {
      Nil // no element downstream if element is deny listed
    } else {
      element :: Nil
    }
  }
}


In [ ]:
fruitsAndDeniedCommands.via(denyFilterFlow).runForeach(println)


In [ ]:
val server = Flow[String].map(_ => "OK")

In [ ]:
val client = Flow[String].merge(Source(List("1","2","3"))).map(s => {println(s); s})

In [ ]:
server.join(client).run()

In [ ]:
import akka.stream._

In [ ]:
val graph = RunnableGraph.fromGraph(GraphDSL.create() { implicit b =>
  import GraphDSL.Implicits._

  val source = Source(List(1,2,3))
  val merge = b.add(Merge[Int](2))
  val bcast = b.add(Broadcast[Int](2))

  source ~> merge ~> Flow[Int].map { s => println(s); s }.filter(_ % 2 == 0) ~> bcast ~> Sink.ignore
            merge                    <~                      bcast
  ClosedShape
})

In [ ]:
graph.run()

In [ ]:
val graph = RunnableGraph.fromGraph(GraphDSL.create() { implicit b =>
  import GraphDSL.Implicits._

  val source = Source(List(1,2,3))
  val merge = b.add(MergePreferred[Int](1))
  val bcast = b.add(Broadcast[Int](2))

  source ~> merge ~> Flow[Int].map { s => println(s); s }.filter(_ % 2 == 0) ~> bcast ~> Sink.ignore
            merge.preferred                   <~                      bcast
  ClosedShape
})

In [ ]:
val g = RunnableGraph.fromGraph(GraphDSL.create() { implicit b =>
  import GraphDSL.Implicits._

  val source = Source(List(1,2,3,4,5))
  val zip = b.add(ZipWith((left: Int, right: Int) => left))
  val bcast = b.add(Broadcast[Int](2))
  val concat = b.add(Concat[Int]())
  val start = Source.single(0)

  source ~> zip.in0
  zip.out/*.map { s => println(s); s }*/ ~> bcast ~> Sink.foreach{println}
  zip.in1 <~ concat <~ start
             concat         <~          bcast
  ClosedShape
})

In [ ]:
g.run()

In [ ]:
val f: Flow[Int, Int, NotUsed] = Flow[Int].map(-_).alsoTo(Sink.foreach{i => println(s"Server: $i")})

In [ ]:
val g = RunnableGraph.fromGraph(GraphDSL.create(f) { implicit b => flow => 
  import GraphDSL.Implicits._

  val source = Source(List(1,2,3,4,5))
  val interleave = b.add(Interleave[Int](2, 1))
  val filter = b.add(Flow[Int].filter(_ > 0))
  val bcast = b.add(Broadcast[Int](2))

  source ~> interleave.in(0)
  flow.out ~> interleave.in(1)
  interleave.out ~> filter ~> bcast ~> flow.in
                              bcast ~> Sink.foreach{i: Int => println(s"Client: $i")}
  ClosedShape
})

In [ ]:
val g = RunnableGraph.fromGraph(GraphDSL.create(f) { implicit b => flow => 
  import GraphDSL.Implicits._

  val source = Source(List(1,2,3,4,5))
  val interleave = b.add(Interleave[Int](2, 1))
  val check = b.add(Flow[Int].map{ i => 
      if (i == -3)  throw new Exception("error")
      else i
  })
  val filter = b.add(Flow[Int].filter(_ > 0))
  val bcast = b.add(Broadcast[Int](2))
  val bcast2 = b.add(Broadcast[Int](2))

  source ~> interleave.in(0)
  flow.out ~> interleave.in(1)
  interleave.out ~> bcast2 ~> Sink.foreach{i : Int => println(s"Client (received): $i")}
                    bcast2 ~> check ~> filter ~> bcast ~> flow.in
                              bcast ~> Sink.foreach{i: Int => println(s"Client: $i")}
  ClosedShape
})

In [ ]:
g.run()

In [ ]:
val r = Source(List(1,2,3)).mapMaterializedValue(_ => 1).to(Sink.ignore)

In [ ]:
val r = Source(List(1,2,3)).alsoTo(Sink.foreach(println)).to(Sink.ignore)

In [ ]:
val r = Source(List(1,2,3)).alsoTo(Sink.foreach(println)).to(Sink.cancelled)

In [ ]:
r.run()

In [ ]:
val countingSrc = Source(Stream.from(1)).delay(1.second, DelayOverflowStrategy.backpressure)
val lastSnk = Sink.last[Int]

val (killSwitch, last) = countingSrc
  .viaMat(KillSwitches.single)(Keep.right)
  .toMat(lastSnk)(Keep.both)
  .run()

In [ ]:
killSwitch.shutdown

In [ ]:
Await.result(last, 60.second)

# The server

In [ ]:
val connections: Source[IncomingConnection, Future[ServerBinding]] =
  Tcp().bind("127.0.0.1", 8890)

In [ ]:
connections.runForeach { connection =>
  println(s"New connection from: ${connection.remoteAddress}")

  val echo = Flow[ByteString]
    .via(Framing.delimiter(ByteString("\n"), maximumFrameLength = 256, allowTruncation = true))
    .map(_.utf8String)
    .map(_ + "!!!\n")
    .merge(Source.single("Hi!"))
    .map( (s: String) => { println(s); ByteString(s)})

  connection.handleWith(echo)
}

In [ ]:
//res6.foreach{println}(ExecutionContext.global)

# The client

In [ ]:
val connclient = Tcp().outgoingConnection("127.0.0.1", 8890)

In [ ]:
import scala.io.StdIn.readLine

val replParser =
  Flow[String].takeWhile(_ != "q").concat(Source.single("BYE")).map(elem => ByteString(s"$elem\n"))

val repl = Flow[ByteString].via(Framing.delimiter(ByteString("\n"), maximumFrameLength = 256, allowTruncation = true)).map(_.utf8String).map(text => println("Server: " + text)).map(_ => readLine("> ")).via(replParser)

val connected = connclient.join(repl).run()